## Import libraries

In [1]:
import pandas as pd


## Read in excel file to pandas

In [2]:
rule_df = pd.read_excel(
    "../data/raw/RiskClassification_Data_Endpoints_V2_Shared.xlsx", sheet_name="RiskRules")

# drop the first coulmn
rule_df = rule_df.iloc[:, 1:]
rule_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
1,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
2,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
3,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
4,No Authentication,Injections,Fail,Anywhere,Anyone,NaN,NaN,Imminent
...,...,...,...,...,...,...,...,...
70,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
71,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
72,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low
73,Some Authentication,All Tests Performed/Available,Pass,Amaricas,Anyone,Yes,Yes,Low


In [3]:
# find the unique values in column "authentication"
auth_col = rule_df["authentication"].unique()
print(f'Unique values in column "authentication": {auth_col}')

# find the unique values in column "security_test_category"
sec_test_col = rule_df["security_test_category"].unique()
print(f'Unique values in column "security_test_category": {sec_test_col}')

# find the unique values in column "security_test_result"
sec_test_res_col = rule_df["security_test_result"].unique()
print(f'Unique values in column "security_test_result": {sec_test_res_col}')

# find the unique values in column "server_location"
server_loc_col = rule_df["server_location"].unique()
print(f'Unique values in column "server_location": {server_loc_col}')

# find the unique values in column "hosting_isp"
host_isp_col = rule_df["hosting_isp"].unique()
print(f'Unique values in column "hosting_isp": {host_isp_col}')

# find the unique values in column "PII"
pii_col = rule_df["PII"].unique()
print(f'Unique values in column "PII": {pii_col}')

# find the unique values in column "FII"
fii_col = rule_df["FII"].unique()
print(f'Unique values in column "FII": {fii_col}')

# find the unique values in column "Risk_Label"
risk_label_col = rule_df["Risk_Label"].unique()
print(f'Unique values in column "Risk_Label": {risk_label_col}')


Unique values in column "authentication": ['No Authentication' 'Some Authentication']
Unique values in column "security_test_category": ['Injections' 'Broken Authentication' 'No Test Performed/Available'
 'All Tests Performed/Available' 'Buffer Overflow' 'XML External Enteties'
 'Cross-Site Scripting']
Unique values in column "security_test_result": ['Fail' nan 'Pass']
Unique values in column "server_location": ['Anywhere' 'Russia' 'China' 'Others' 'West Europe' 'Amaricas']
Unique values in column "hosting_isp": ['Anyone']
Unique values in column "PII": [nan 'Yes' 'No']
Unique values in column "FII": [nan 'Yes' 'No']
Unique values in column "Risk_Label": ['Imminent' 'High' 'Medium' 'Low']


In [4]:
api_df = pd.read_excel("../data/raw/RiskClassification_Data_Endpoints_V2.xlsx", "Core_Endpoint",
                       usecols="A:R")
# rename column "security_test_result (FALSE=Passed; TRUE=Failed)" to "security_test_result"
api_df.rename(columns={
              'security_test_result (FALSE=Passed; TRUE=Failed)': 'security_test_result'}, inplace=True)
api_df = api_df[['authentication', 'security_test_category', 'security_test_result',
                    'server_location', 'hosting_isp']]
# add column "PII" and "FII" and fill with "No"
api_df["PII"] = "Yes"
api_df["FII"] = "No"
api_df.head()

,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,header,NaN,NaN,United States,OVH Hosting,Yes,No
1,NaN,NaN,NaN,NaN,NaN,Yes,No
2,header,NaN,NaN,Singapore,Incapsula Inc,Yes,No
3,header,NaN,NaN,United States,"Akamai Technologies, Inc.",Yes,No
4,query,NaN,NaN,Russia,Yandex enterprise network,Yes,No


In [5]:
api_df["authentication"].unique()

array(['header', nan, 'query', 'BasicAuth', 'none', 'OAuth2', 'JWT',
       'path'], dtype=object)

In [6]:
# process column authentication from api_df, replace nan and none with "No Authentication"
api_df["authentication"] = api_df["authentication"].fillna("No Authentication")
api_df["authentication"] = api_df["authentication"].replace(
    "None", "No Authentication")
# replace all value that not "No Authentication" with "Some Authentication"
api_df["authentication"].mask(api_df["authentication"] != "No Authentication", "Some Authentication", inplace=True)
api_df.head()


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,Some Authentication,NaN,NaN,United States,OVH Hosting,Yes,No
1,No Authentication,NaN,NaN,NaN,NaN,Yes,No
2,Some Authentication,NaN,NaN,Singapore,Incapsula Inc,Yes,No
3,Some Authentication,NaN,NaN,United States,"Akamai Technologies, Inc.",Yes,No
4,Some Authentication,NaN,NaN,Russia,Yandex enterprise network,Yes,No


In [7]:
api_df["security_test_category"].unique()

array([nan, 'Broken Authentication', 'Buffer Overflow',
       'Insecure Deserialization', 'SQL Injection',
       'Cross-Site Scripting', 'XML External Entities'], dtype=object)

In [8]:
# process column security_test_category from api_df, replace nan with "No Test Performed/Available"
api_df["security_test_category"] = api_df["security_test_category"].fillna(
    "No Test Performed/Available")
# replace Injections with "SQL Injection"
api_df["security_test_category"] = api_df["security_test_category"].replace(
    "Injections", "SQL Injection")
# replace "Insecure Deserialization" with "No Test Performed/Available"
api_df["security_test_category"] = api_df["security_test_category"].replace(
    "Insecure Deserialization", "No Test Performed/Available")
api_df.head()


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,Some Authentication,No Test Performed/Available,NaN,United States,OVH Hosting,Yes,No
1,No Authentication,No Test Performed/Available,NaN,NaN,NaN,Yes,No
2,Some Authentication,No Test Performed/Available,NaN,Singapore,Incapsula Inc,Yes,No
3,Some Authentication,No Test Performed/Available,NaN,United States,"Akamai Technologies, Inc.",Yes,No
4,Some Authentication,No Test Performed/Available,NaN,Russia,Yandex enterprise network,Yes,No


In [9]:
api_df["security_test_result"].unique()


array([nan,  0.,  1.])

In [10]:
# process column security_test_result from api_df, replace false with "Passed"
api_df["security_test_result"] = api_df["security_test_result"].replace(
    0., "Pass")
# replace true with "Failed"
api_df["security_test_result"] = api_df["security_test_result"].replace(
    1., "Fail")
api_df.head()


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,Some Authentication,No Test Performed/Available,NaN,United States,OVH Hosting,Yes,No
1,No Authentication,No Test Performed/Available,NaN,NaN,NaN,Yes,No
2,Some Authentication,No Test Performed/Available,NaN,Singapore,Incapsula Inc,Yes,No
3,Some Authentication,No Test Performed/Available,NaN,United States,"Akamai Technologies, Inc.",Yes,No
4,Some Authentication,No Test Performed/Available,NaN,Russia,Yandex enterprise network,Yes,No


In [11]:
# ['Anywhere' 'Russia' 'China' 'Others' 'West Europe' 'Amaricas']
api_df["server_location"].unique()


array(['United States', nan, 'Singapore', 'Russia', 'Australia', 'Canada',
       'United Kingdom', 'India', 'Czechia', 'Ireland', 'Japan',
       'Germany', 'Spain', 'Lithuania'], dtype=object)

In [14]:
# process column server_location from api_df, replace nan with "Anywhere"
api_df["server_location"] = api_df["server_location"].fillna("Anywhere")
api_df["server_location"] = api_df["server_location"].replace(
    ["United States", "Canada"], "Amaricas")
api_df["server_location"] = api_df["server_location"].replace(
    ["United Kingdom", "Ireland", "Germany", "Spain"], "West Europe")
# replace everything else with "Others"
api_df["server_location"] = api_df["server_location"].replace(
    ["India", "Japan", "Australia",  "Czechia", "Lithuania", "Singapore"], "Others")
api_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,Some Authentication,No Test Performed/Available,NaN,Amaricas,OVH Hosting,Yes,No
1,No Authentication,No Test Performed/Available,NaN,Anywhere,NaN,Yes,No
2,Some Authentication,No Test Performed/Available,NaN,Others,Incapsula Inc,Yes,No
3,Some Authentication,No Test Performed/Available,NaN,Amaricas,"Akamai Technologies, Inc.",Yes,No
4,Some Authentication,No Test Performed/Available,NaN,Russia,Yandex enterprise network,Yes,No
...,...,...,...,...,...,...,...
108,Some Authentication,Buffer Overflow,Pass,Amaricas,"Amazon.com, Inc.",Yes,No
109,Some Authentication,XML External Entities,Pass,Amaricas,"Amazon.com, Inc.",Yes,No
110,Some Authentication,No Test Performed/Available,NaN,Amaricas,"Cloudflare, Inc.",Yes,No
111,Some Authentication,Buffer Overflow,Pass,Amaricas,"Amazon.com, Inc.",Yes,No


In [116]:
# process column hosting_isp from api_df, replace value with "Anyone"
api_df["hosting_isp"] = "Anyone"
api_df.head()

,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII
0,Some Authentication,No Test Performed/Available,NaN,Amaricas,Anyone,No,Yes
1,No Authentication,No Test Performed/Available,NaN,Anywhere,Anyone,No,Yes
2,Some Authentication,No Test Performed/Available,NaN,Others,Anyone,No,Yes
3,Some Authentication,No Test Performed/Available,NaN,Amaricas,Anyone,No,Yes
4,Some Authentication,No Test Performed/Available,NaN,Russia,Anyone,No,Yes


In [117]:
# slice row 32 of rule_df
# rule_df = rule_df.iloc[[32]]
# rule_df


In [15]:

# write the rule to classify the Risk_Label based on rule_df
def classify_risk(row):
    """
    Check if the row is the same with any row in rule_df
    """
    # loop through each row in rule_df
    for i in range(len(rule_df)):
        row_copy = rule_df.iloc[i]
        # drop the column "Risk_Label"
        row_copy = row_copy.drop(["Risk_Label"])
        if row_copy['server_location'] == 'Anywhere':
            row_copy['server_location'] = row['server_location']
        if row_copy['security_test_category'] == 'All Tests Performed/Available':
            row_copy['security_test_category'] = row['security_test_category']
        # check if the row is the same with any row in rule_df
        if row_copy.equals(row):
            # return the value in column "Risk_Label"
            return rule_df.iloc[i]["Risk_Label"]
    return "Low Risk"


In [16]:
# apply the function classify_risk to each row in api_df
api_df["Risk_Label"] = api_df.apply(classify_risk, axis=1)
api_df

,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,Some Authentication,No Test Performed/Available,NaN,Amaricas,OVH Hosting,Yes,No,Low Risk
1,No Authentication,No Test Performed/Available,NaN,Anywhere,NaN,Yes,No,Low Risk
2,Some Authentication,No Test Performed/Available,NaN,Others,Incapsula Inc,Yes,No,Low Risk
3,Some Authentication,No Test Performed/Available,NaN,Amaricas,"Akamai Technologies, Inc.",Yes,No,Low Risk
4,Some Authentication,No Test Performed/Available,NaN,Russia,Yandex enterprise network,Yes,No,Low Risk
...,...,...,...,...,...,...,...,...
108,Some Authentication,Buffer Overflow,Pass,Amaricas,"Amazon.com, Inc.",Yes,No,Low Risk
109,Some Authentication,XML External Entities,Pass,Amaricas,"Amazon.com, Inc.",Yes,No,Low Risk
110,Some Authentication,No Test Performed/Available,NaN,Amaricas,"Cloudflare, Inc.",Yes,No,Low Risk
111,Some Authentication,Buffer Overflow,Pass,Amaricas,"Amazon.com, Inc.",Yes,No,Low Risk


In [120]:
# save the result to excel
api_df.to_excel("../data/processed/RiskLabel_YesNo.xlsx", index=False)

In [18]:
api_df


,authentication,security_test_category,security_test_result,server_location,hosting_isp,PII,FII,Risk_Label
0,Some Authentication,No Test Performed/Available,NaN,Amaricas,Anyone,No,No,Medium
1,No Authentication,No Test Performed/Available,NaN,Anywhere,Anyone,No,No,No Match
2,Some Authentication,No Test Performed/Available,NaN,Others,Anyone,No,No,Medium
3,Some Authentication,No Test Performed/Available,NaN,Amaricas,Anyone,No,No,Medium
4,Some Authentication,No Test Performed/Available,NaN,Russia,Anyone,No,No,Medium
...,...,...,...,...,...,...,...,...
108,Some Authentication,Buffer Overflow,Pass,Amaricas,Anyone,No,No,No Match
109,Some Authentication,XML External Entities,Pass,Amaricas,Anyone,No,No,No Match
110,Some Authentication,No Test Performed/Available,NaN,Amaricas,Anyone,No,No,Medium
111,Some Authentication,Buffer Overflow,Pass,Amaricas,Anyone,No,No,No Match
